In [1]:
import Bio
import os
import pandas as pd
from Bio import SeqIO

# Define the directory containing your FASTA files
input_dir = 'mitimpair_codons'

# Define the output directory to save the filtered files
output_dir = 'mitimpair_codons_filtered'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Iterate over all files in the input directory
for filename in os.listdir(input_dir):
    # Process only FASTA files (adjust the extensions if necessary)
    if filename.endswith('.fasta') or filename.endswith('.fa') or filename.endswith('.fas'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, filename)
        
        # Read all sequences from the input FASTA file
        with open(input_filepath, 'r') as input_handle:
            sequences = list(SeqIO.parse(input_handle, 'fasta'))
        
        # Filter out sequences whose IDs start with 'en'
        filtered_sequences = [seq for seq in sequences if not seq.id.startswith('en')]
        
        # Write the filtered sequences to the output file
        with open(output_filepath, 'w') as output_handle:
            SeqIO.write(filtered_sequences, output_handle, 'fasta')
        
        # Print a summary
        num_removed = len(sequences) - len(filtered_sequences)
        print(f"Processed {filename}: {num_removed} sequences removed.")


Processed EAW70196.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001153583.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001177793.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001179.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001184044.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001186768.2_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001188.1_bat_only_aligned_nucleotide.fasta: 0 sequences removed.
Processed NP_001274364.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001275684.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001278360.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001289201.1_bat_only_aligned_nucleotide.fasta: 1 sequences removed.
Processed NP_001304107.1_bat_only_aligned_nucleotide.fasta: 0 sequences removed.
Processed NP_001308657.1_bat_only_alig

In [2]:
from ete3 import Tree
from Bio import SeqIO
import os

# Paths
full_tree_file = "bat_tags_noen.nwk"  # Path to the full tree file
pal2nal_dir = "mitimpair_pal2nal_noen"  # Directory with cleaned codon-aligned files
output_tree_dir = "mitimpair_trees_noen"  # Directory to store pruned trees

# Species tag to full name mapping
tag_to_species = {
    "rf": "Rhinolophus_ferrumequinum",
    "dr": "Desmodus_rotundus",
    "ra": "Rousettus_aegyptiacus",
    "pv": "Pteropus_vampyrus",
    "pg": "Pteropus_giganteus",
    "pa": "Pteropus_alecto",
    "pk": "Pipistrellus_kuhlii",
    "ph": "Phyllostomus_hastatus",
    "pd": "Phyllostomus_discolor",
    "mm": "Myotis_myotis",
    "ml": "Myotis_lucifugus",
    "mb": "Myotis_brandtii",
    "mo": "Molossus_molossus",
    "md": "Myotis_daubentonii",
    "sb": "Saccopteryx_bilineata",
    "aj": "Artibeus_jamaicensis",
    "pn": "Pipistrellus_nathusii",
    "en": "Eptesicus_nilssonii",
}

# Create output directory if it doesn't exist
os.makedirs(output_tree_dir, exist_ok=True)

# Load the full tree
full_tree = Tree(full_tree_file, format=1)

# Process each alignment file
for alignment_file in os.listdir(pal2nal_dir):
    if alignment_file.endswith(".fasta"):
        alignment_path = os.path.join(pal2nal_dir, alignment_file)

        # Check if the file is empty
        if os.stat(alignment_path).st_size == 0:
            print(f"Skipping empty file: {alignment_file}")
            continue

        print(f"\nProcessing: {alignment_file}")

        # Extract sequence names from the alignment
        sequence_names = {}
        for record in SeqIO.parse(alignment_path, "fasta"):
            tag = record.id[:2]  # Extract the first two letters of the record ID
            if tag in tag_to_species:
                species_name = tag_to_species[tag]
                sequence_names[species_name] = record.id
            else:
                print(f"Warning: Unrecognized tag '{tag}' in {record.id}")

        # Debugging: Print extracted sequence names
        print(f"Extracted sequence names for {alignment_file}:")
        for species, seq_id in sequence_names.items():
            print(f"  {species}: {seq_id}")

        # Copy the full tree for modification
        unique_tree = full_tree.copy()

        # Rename leaves to match sequence names and remove unmatched leaves
        unmatched_leaves = []
        for leaf in unique_tree.get_leaves():
            if leaf.name in sequence_names:
                leaf.name = sequence_names[leaf.name]  # Rename leaf to match sequence ID
            else:
                unmatched_leaves.append(leaf.name)

        # Debugging: Print unmatched leaves
        if unmatched_leaves:
            print(f"Unmatched leaves for {alignment_file}: {unmatched_leaves}")

        # Remove unmatched leaves
        for leaf_name in unmatched_leaves:
            leaf = unique_tree.search_nodes(name=leaf_name)[0]
            leaf.detach()

        # Save the unique tree
        output_tree_path = os.path.join(output_tree_dir, f"{alignment_file}_pruned.nwk")
        unique_tree.write(outfile=output_tree_path, format=1)
        print(f"Unique tree saved to: {output_tree_path}")


Processing: EAW70196.1_bat_only_aligned_nucleotide_codon_aligned.fasta
Extracted sequence names for EAW70196.1_bat_only_aligned_nucleotide_codon_aligned.fasta:
  Pipistrellus_nathusii: pnCAK6448204.1
  Rhinolophus_ferrumequinum: rfXP_032967603.1
  Molossus_molossus: moXP_036110306.1
  Saccopteryx_bilineata: sbXP_066089324.1
  Myotis_myotis: mmXP_036174461.1
  Myotis_daubentonii: mdXP_059558664.1
  Myotis_brandtii: mbXP_005873631.1
  Myotis_lucifugus: mlXP_006082442.1
  Pipistrellus_kuhlii: pkXP_036276778.1
  Rousettus_aegyptiacus: raXP_015986601.1
  Pteropus_vampyrus: pvXP_023391721.1
  Pteropus_giganteus: pgXP_039700286.1
  Pteropus_alecto: paXP_006921389.1
  Desmodus_rotundus: drXP_024419726.2
  Artibeus_jamaicensis: ajXP_037023424.2
  Phyllostomus_hastatus: phXP_045703543.1
  Phyllostomus_discolor: pdXP_028365372.1
Unique tree saved to: mitimpair_trees_noen\EAW70196.1_bat_only_aligned_nucleotide_codon_aligned.fasta_pruned.nwk
Skipping empty file: NP_001153583.1_bat_only_aligned_nuc

In [3]:
import os

# Define the directory containing your .log files
log_dir = 'mitimpair_busted_noen'

# Get a list of all .log files in the directory
log_files = [f for f in os.listdir(log_dir) if f.endswith('.log')]

# Iterate over each log file
for log_file in log_files:
    file_path = os.path.join(log_dir, log_file)
    print(log_file)
    # Read the file and get the last 2 lines
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        if len(lines) >= 2:
            last_two_lines = lines[-2:]  # Get the last two lines
        else:
            last_two_lines = lines  # If the file has fewer than 2 lines
        for line in last_two_lines:
            print(line.rstrip())
    print('-' * 80)  # Separator line


EAW70196.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
## Branch-site unrestricted statistical test of episodic diversification [BUSTED]
Likelihood ratio test for episodic diversifying positive selection, **p =   0.0259**.
--------------------------------------------------------------------------------
NP_001177793.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log

Check errors.log for execution error details.
--------------------------------------------------------------------------------
NP_001179.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
model-averaged inference, or rerunning the alignment with data at that
site masked to confirm robustness of the result.
--------------------------------------------------------------------------------
NP_001184044.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
## Branch-site unrestricted statistical test of episodic diversification [BUSTED]
Likelihood ratio test for episodic diversifying pos

In [ ]:
import os

# Define the directory containing your log files
log_dir = 'mitimpair_absrel_noen'

# Iterate over all files in the directory
for filename in os.listdir(log_dir):
    if filename.endswith('.log'):
        filepath = os.path.join(log_dir, filename)
        with open(filepath, 'r') as file:
            start_printing = False
            for line in file:
                if '### Adaptive branch site random effects likelihood test' in line:
                    start_printing = True
                    print(filename)  # Print the filename
                    print(line.strip())  # Print the starting line
                    continue
                if start_printing:
                    print(line.rstrip())
            if not start_printing:
                print(f"{filename} does not contain the target section.")
            # Add a separator between files
            print('-' * 80)


In [1]:
import os
import re

# Define the directory containing your .log files
log_dir = 'mitimpair_bustede1'  # Update this if your directory name is different

# Output file to store the protein names
output_file = 'mitimpair_bustede1_noen_prots_nofdr.txt'

# Open the output file in write mode
with open(output_file, 'w') as out_f:

    # Iterate over all .log files in the directory
    for filename in os.listdir(log_dir):
        if filename.endswith('.log'):
            filepath = os.path.join(log_dir, filename)
            # Extract the protein name from the filename based on the rules
            # Split the filename by '_'
            parts = filename.split('_')
            if parts[0].startswith('NP') or parts[0].startswith('XP'):
                # If the protein name starts with 'NP' or 'XP', join the first two parts
                protein_name = '_'.join(parts[:2])
            else:
                # Otherwise, the protein name is the first part
                protein_name = parts[0]

            # Open and read the log file
            with open(filepath, 'r') as log_f:
                lines = log_f.readlines()
                for line in lines:
                    if 'Likelihood ratio test for episodic diversifying positive selection' in line:
                        # Extract the p-value from this line
                        match = re.search(r'p =\s*([\d\.eE+-]+)', line)
                        if match:
                            p_value_str = match.group(1)
                            try:
                                p_value = float(p_value_str)
                                if p_value < 0.05:
                                    # Write the protein name to the output file
                                    out_f.write(protein_name + '\n')
                                    print(f"{protein_name}: p-value {p_value} < 0.05, added to {output_file}")
                                else:
                                    print(f"{protein_name}: p-value {p_value} >= 0.05, not added")
                            except ValueError:
                                print(f"{protein_name}: Could not convert p-value '{p_value_str}' to float")
                        else:
                            print(f"{protein_name}: p-value not found in line: {line.strip()}")
                        # Break after processing the line
                        break
                else:
                    # If we didn't find the line with 'Likelihood ratio test...'
                    print(f"{protein_name}: 'Likelihood ratio test' line not found in {filename}")



EAW70196.1: p-value 0.4824 >= 0.05, not added
NP_001177793.1: 'Likelihood ratio test' line not found in NP_001177793.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001179.1: p-value 0.3714 >= 0.05, not added
NP_001184044.1: p-value 0.5 >= 0.05, not added
NP_001186768.2: p-value 0.5 >= 0.05, not added
NP_001274364.1: p-value 0.484 >= 0.05, not added
NP_001275684.1: p-value 0.5 >= 0.05, not added
NP_001278360.1: p-value 0.5 >= 0.05, not added
NP_001289201.1: p-value 0.5 >= 0.05, not added
NP_001308657.1: p-value 0.4973 >= 0.05, not added
NP_001309749.1: p-value 0.304 >= 0.05, not added
NP_001310513.1: 'Likelihood ratio test' line not found in NP_001310513.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001317309.1: 'Likelihood ratio test' line not found in NP_001317309.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001339859.1: p-value 0.3713 >= 0.05, not added
NP_001341754.1: p-value 0.5 >= 0.05, not added
NP_001352690.1: p-value 0.4975 

In [3]:
%pip install statsmodels


   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.8 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.8 MB 7.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.8 MB 8.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 8.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 8.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.1/9.8 MB 8.4 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.8 MB 1.9 MB/s eta 0:00:04
   ------------- -------------------------- 3.4/9.8 MB 2.2 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.8 MB 2.4 MB/s eta 0:00:03
   ---------------------- ----------------- 5.5/9.8 MB 2.8 MB/s eta 0:00:02
   ---------------------------- ----------- 7.1/9.8 MB 3.3 MB/s eta 0:00:01
   ------------------------------------ --- 8.9/9.8 MB 3.8 MB/s eta 0:00:01
   -----------------------

In [4]:
#!/usr/bin/env python3

import os
import re
from statsmodels.stats.multitest import multipletests

# Directory containing your .log files
log_dir = 'mitimpair_bustede1'

# Output file
output_file = 'mitimpair_bustede1_noen_prots.txt'

# We will store all (protein_name, p_value) in a list for later correction
pvals_data = []

# 1. Parse each .log file and extract p-values
for filename in os.listdir(log_dir):
    if filename.endswith('.log'):
        filepath = os.path.join(log_dir, filename)
        # Extract the protein name from the filename
        parts = filename.split('_')
        if parts[0].startswith('NP') or parts[0].startswith('XP'):
            protein_name = '_'.join(parts[:2])
        else:
            protein_name = parts[0]

        # Read the log file
        with open(filepath, 'r') as log_f:
            lines = log_f.readlines()
            found_pval = False
            for line in lines:
                if 'Likelihood ratio test for episodic diversifying positive selection' in line:
                    # Extract the p-value
                    match = re.search(r'p =\s*([\d\.eE+-]+)', line)
                    if match:
                        p_value_str = match.group(1)
                        try:
                            p_value = float(p_value_str)
                            # Store this (protein, p-value)
                            pvals_data.append((protein_name, p_value))
                            found_pval = True
                        except ValueError:
                            print(f"{protein_name}: Could not convert p-value '{p_value_str}' to float")
                    else:
                        print(f"{protein_name}: p-value not found in line: {line.strip()}")
                    break  # Stop after finding the line
            if not found_pval:
                print(f"{protein_name}: 'Likelihood ratio test' line not found or no p-value in {filename}")

# 2. Once we've collected all p-values, apply FDR correction
p_values = [x[1] for x in pvals_data]  # just the numeric p-values

if len(p_values) == 0:
    print("No p-values found. Exiting.")
    exit()

# Benjamini-Hochberg procedure (FDR)
# returns: reject (boolean array), pvals_corrected, alphacSidak, alphacBonf
reject_array, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

# 3. Write results to file (only those that pass FDR, i.e., q < 0.05)
with open(output_file, 'w') as out_f:
    for i, (protein_name, original_pval) in enumerate(pvals_data):
        fdr_pval = pvals_corrected[i]
        if reject_array[i]:
            # This means fdr_pval < 0.05
            out_f.write(protein_name + '\n')
            print(f"{protein_name}: raw p = {original_pval:.4g}, FDR p = {fdr_pval:.4g} (SIGNIFICANT)")
        else:
            print(f"{protein_name}: raw p = {original_pval:.4g}, FDR p = {fdr_pval:.4g} (not significant)")


NP_001177793.1: 'Likelihood ratio test' line not found or no p-value in NP_001177793.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001310513.1: 'Likelihood ratio test' line not found or no p-value in NP_001310513.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001317309.1: 'Likelihood ratio test' line not found or no p-value in NP_001317309.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
NP_001365933.1: 'Likelihood ratio test' line not found or no p-value in NP_001365933.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
O75431.1: 'Likelihood ratio test' line not found or no p-value in O75431.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
Q8IXM3.1: 'Likelihood ratio test' line not found or no p-value in Q8IXM3.1_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
Q96GC5.2: 'Likelihood ratio test' line not found or no p-value in Q96GC5.2_bat_only_aligned_nucleotide_codon_aligned.fasta_busted.log
Q9UH62.1: 'Lik

In [6]:
import os

# Define the directory containing your log files
log_dir = 'mitimpair_absrel_noen'

# Define the path to the protein names file
protein_names_file = 'mitimpair_busted_noen_prots.txt'

# Read the protein names into a set for efficient lookup
with open(protein_names_file, 'r') as f:
    protein_names = set(line.strip() for line in f if line.strip())

# Iterate over all files in the directory
for filename in os.listdir(log_dir):
    if filename.endswith('.log'):
        # Extract the protein name from the filename based on the rules
        parts = filename.split('_')
        if parts[0].startswith('NP') or parts[0].startswith('XP'):
            # If the protein name starts with 'NP' or 'XP', join the first two parts
            protein_name = '_'.join(parts[:2])
        else:
            # Otherwise, the protein name is the first part
            protein_name = parts[0]

        # Check if the protein name is in the set
        if protein_name in protein_names:
            filepath = os.path.join(log_dir, filename)
            with open(filepath, 'r') as file:
                start_printing = False
                for line in file:
                    if '### Adaptive branch site random effects likelihood test' in line:
                        start_printing = True
                        print(filename)  # Print the filename
                        print(line.strip())  # Print the starting line
                        continue
                    if start_printing:
                        print(line.rstrip())
                if not start_printing:
                    print(f"{filename} does not contain the target section.")
                # Add a separator between files
                print('-' * 80)
        else:
            # If the protein name is not in the list, skip the file
            continue


EAW70196.1_bat_only_aligned_nucleotide_codon_aligned.fasta_absrel.log
### Adaptive branch site random effects likelihood test
Likelihood ratio test for episodic diversifying positive selection at Holm-Bonferroni corrected _p =   0.0500_ found **1** branches under selection among **30** tested.

* mlXP_006082442_1, p-value =  0.00000
--------------------------------------------------------------------------------
NP_001179.1_bat_only_aligned_nucleotide_codon_aligned.fasta_absrel.log
### Adaptive branch site random effects likelihood test
Likelihood ratio test for episodic diversifying positive selection at Holm-Bonferroni corrected _p =   0.0500_ found **1** branches under selection among **26** tested.

* pvXP_023394266_1, p-value =  0.00051
--------------------------------------------------------------------------------
NP_001361015.1_bat_only_aligned_nucleotide_codon_aligned.fasta_absrel.log
### Adaptive branch site random effects likelihood test
Likelihood ratio test for episodic di